In [8]:
import numpy as np
import pandas as pd
import redis

In [9]:
hostname = ''
portnumber = 19001
password = ''

r = redis. StrictRedis(host=hostname,
port=portnumber,
password=password)

In [10]:
r.ping()

True

## Loading the data numpy zip

In [11]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,name,role,facial_features
0,Angelina Jolie,Student,"[0.43240866661071775, 1.0746602058410644, -0.1..."
1,Barack Obama,Teacher,"[0.9487081527709961, -0.5583571434020996, 0.09..."
2,Chris Evans,Student,"[0.7200981616973877, -1.4685219764709472, -0.7..."
3,Morgan Freeman,Teacher,"[0.4854259490966797, -0.6193630218505859, -0.4..."
4,Scarlett Johansson,Student,"[0.19131546020507811, -0.48847036361694335, -1..."


In [12]:
df['name_role']= df['name']+'@'+df['role']
df

,name,role,facial_features,name_role
0,Angelina Jolie,Student,"[0.43240866661071775, 1.0746602058410644, -0.1...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.9487081527709961, -0.5583571434020996, 0.09...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7200981616973877, -1.4685219764709472, -0.7...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.4854259490966797, -0.6193630218505859, -0.4...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19131546020507811, -0.48847036361694335, -1...",Scarlett Johansson@Student


In [13]:
records = df[ ['name_role' ,'facial_features' ]]. to_dict(orient='records' )
len(records)

5

## saving the data in redis
- redis hashes
- keyname = 'academy:register'

In [14]:
for record in records:
    name_role = record['name_role' ]
    vector = record['facial_features' ]
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role, value=vector_bytes)
    

### retrive records from redis

In [15]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Angelina Jolie@Student'        b'ffff\x95\xac\xdb?fff\xe6\xce1\xf1?3333xx\xc0...
b'Morgan Freeman@Teacher'        b'\x00\x00\x00\x008\x11\xdf?ffff\xd2\xd1\xe3\x...
b'Scarlett Johansson@Student'    b'ffff\x06}\xc8?3333\x19C\xdf\xbf\x9a\x99\x99\...
b'Tamim@Student'                 b'\x1a\x0c\x19?\xe3\xcd\x96\xbf\xab\x9a\x12\xc...
b'Barack Obama@Teacher'          b'3333\xd1[\xee?\xcd\xcc\xcc\xcc\x0f\xde\xe1\x...
b'Chris Evans@Student'           b'\xcd\xcc\xccL\x0b\x0b\xe7?fff\xe6\x10\x7f\xf...
dtype: object

In [16]:
#np frombuffer(retrive_ series[b "Morgan Freeman@Teacher' dtype=np. float32). shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [17]:
retrive_series.index = index

In [18]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Angelina Jolie@Student,"[2.720083e+23, 1.7162043, -2.720083e+23, 1.884..."
1,Morgan Freeman@Teacher,"[0.0, 1.742713, 2.720083e+23, -1.7798407, 2.72..."
2,Scarlett Johansson@Student,"[2.720083e+23, 1.5663154, 4.172325e-08, -1.744..."
3,Tamim@Student,"[0.5978409, -1.1781582, -2.2906902, -0.5149337..."
4,Barack Obama@Teacher,"[4.172325e-08, 1.862177, -107374184.0, -1.7645..."
5,Chris Evans@Student,"[107374184.0, 1.8050245, -2.720083e+23, -1.933..."


### Registeration Form

In [19]:
import cv2
from insightface.app import FaceAnalysis

C:\Users\Tamim\Desktop\FYP\Notes\attendance_system\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [20]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
    root='insightface_model',
    providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# we should notset det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### 1. colletion of person name and role

In [24]:
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose:
    1. Student
    2. Teacher
    
    Enter number either 1 or 2: """)
    
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry, try again.')
        if i == trials - 1:
            print('Exceeds maximum trials.')
            role = 'Unknown'
            break

key = person_name + '@' + role
print('Your name =', person_name)
print('Your role =', role)
print('Key =', key)


Enter your name:  tamo

    Please choose:
    1. Student
    2. Teacher
    
    Enter number either 1 or 2:  1


Your name = tamo
Your role = Student
Key = tamo@Student


### 2.collection of samples

In [25]:
cap = cv2.VideoCapture(0)  # 0 for default camera, 1 for external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print('Unable to read camera')
        break
    # get results from the insightface model
    results = faceapp.get(frame, max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
        
        # extract facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
    
    if sample >= 200:
        break
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):  # triggers only when 'q' is pressed on the keyboard
        break

cap.release()
cv2.destroyAllWindows()

### 3. Optimizing the data and saving into redis

In [26]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [27]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [28]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1